In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import requests
import pandas as pd

In [2]:
def load_SG_GEN_PARL(geoAreaCode):
    assert isinstance(geoAreaCode, int)
    url = f'https://unstats.un.org/sdgapi/v1/sdg/Series/SG_GEN_PARL/GeoArea/{geoAreaCode}/DataSlice'
    response = requests.get(url)
    data = response.json()['dimensions']
    year = [int(data[i]['timePeriodStart']) for i in range(len(response.json()['dimensions']))]
    prop_seats_women = [float(data[i]['value']) for i in range(len(response.json()['dimensions']))]
    return pd.DataFrame(data={'year': year, 'prop_seats_women': prop_seats_women})

In [3]:
# testing the function
display(load_SG_GEN_PARL(1).head())

,year,prop_seats_women
0,2003,15.12
1,2007,17.14
2,2004,15.21
3,2005,15.91
4,2001,13.79


In [4]:
def load_geoArea_options():
    url = f'https://unstats.un.org/sdgapi/v1/sdg/GeoArea/List'
    response = requests.get(url)
    options = response.json()
    for opt in options:
        opt['label'] = opt.pop('geoAreaName')
        opt['value'] = opt.pop('geoAreaCode')
    return options

geoArea_options = load_geoArea_options()

In [5]:
# testing the function and creating the options
geoArea_options = load_geoArea_options()
geoArea_options[:10]

[{'label': 'Afghanistan', 'value': '4'},
 {'label': 'Åland Islands', 'value': '248'},
 {'label': 'Albania', 'value': '8'},
 {'label': 'Algeria', 'value': '12'},
 {'label': 'American Samoa', 'value': '16'},
 {'label': 'Andorra', 'value': '20'},
 {'label': 'Angola', 'value': '24'},
 {'label': 'Anguilla', 'value': '660'},
 {'label': 'Antarctica', 'value': '10'},
 {'label': 'Antigua and Barbuda', 'value': '28'}]

In [6]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='My Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(options=geoArea_options, 
                 value='1', 
                 id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    df = load_SG_GEN_PARL(geoAreaCode=int(value))
    return px.scatter(df, x='year', y='prop_seats_women')

if __name__ == '__main__':
    app.run_server(debug=False, port=8049, use_reloader=False)

[2023-06-26 10:33:23,735] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/abrasm/.local/share/virtualenvs/BDPP-BtGq7L5S/lib/python3.10/site-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/abrasm/.local/share/virtualenvs/BDPP-BtGq7L5S/lib/python3.10/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/abrasm/.local/share/virtualenvs/BDPP-BtGq7L5S/lib/python3.10/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/abrasm/.local/share/virtualenvs/BDPP-BtGq7L5S/lib/python3.10/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/abrasm/.local/share/virtualenvs/BDPP-BtGq7L5S/lib/python3.10/site-packages/dash/dash.py", line 1258, in dispatch
    ctx.run(
 